# Load Data

In [0]:
import pyspark.sql.functions as F
from functools import reduce
from pyspark.sql import Window

In [0]:
#### file paths
# delta table paths for geo_df, pin_df and user_df
delta_base_path = "/mnt/pinterest_data/delta_tables/"

#### Paths for raw/original data
raw_delta_geo_path = delta_base_path + "raw/geo"
raw_delta_pin_path = delta_base_path + "raw/pin"
raw_delta_user_path = delta_base_path + "raw/user"

#### Paths for transformed/processed data
transformed_delta_geo_path = delta_base_path + "transformed/geo"
transformed_delta_pin_path = delta_base_path + "transformed/pin"
transformed_delta_user_path = delta_base_path + "transformed/user"

In [0]:
# load the dataframes and check schema
df_pin = spark.read.format("delta").load(transformed_delta_pin_path)
df_geo = spark.read.format("delta").load(transformed_delta_geo_path)
df_user = spark.read.format("delta").load(transformed_delta_user_path)

df_pin.printSchema()
df_geo.printSchema()
df_user.printSchema()

root
-- ind: long (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: struct (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
-- timestamp: timestamp (nullable = true)

root
-- ind: long (nullable = true)
-- user_name: string (nullable = true)
-- age: long (nullable = true)
-- date_joined: timestamp (nullable = true)

In [0]:
# create views of the three dataframes to run sql queries on it
df_geo.createOrReplaceTempView("view_geo")
df_pin.createOrReplaceTempView("view_pin")
df_user.createOrReplaceTempView("view_user")

# Find the most popular category in each country

## Spark

In [0]:
# create a window function that is partitioned by 'country' and ordered in descending order
window = Window.partitionBy("country").orderBy(F.desc("count"))

# join pinterest and geolocation data, group by country and category and then count the instances, then create a ranking column over the window function, filter by the first rank, drop the rank column and rename count column.
popular_category_in_each_country_spark = (df_pin.join(
    df_geo,
    on=df_pin.ind == df_geo.ind,
    how="inner"
)
.groupBy(
    "country", "category"
)
.agg(F.count("*").alias("count"))
.withColumn(
    "rank",
    F.rank().over(window)
)
.filter(F.col("rank") == 1)
.drop("rank")
.withColumnRenamed("count", "category_count")
)

In [0]:
display(popular_category_in_each_country_spark)

country,category,category_count
Afghanistan,education,94
Albania,art,193
Algeria,quotes,256
American Samoa,tattoos,98
Andorra,tattoos,77
Angola,education,30
Anguilla,christmas,25
Antarctica (the territory South of 60 deg S),christmas,36
Antigua and Barbuda,travel,29
Argentina,tattoos,72


## SQL

In [0]:
%sql
select country, category, category_count
from(
  select country, 
       category, 
       count(*) as category_count,
       rank() over(partition by country order by count(*) desc) as rank
  from view_pin inner join view_geo
  on view_pin.ind == view_geo.ind
  group by country, category
) tmp
where rank = 1

country,category,category_count
Afghanistan,education,94
Albania,art,193
Algeria,quotes,256
American Samoa,tattoos,98
Andorra,tattoos,77
Angola,education,30
Anguilla,christmas,25
Antarctica (the territory South of 60 deg S),christmas,36
Antigua and Barbuda,travel,29
Argentina,tattoos,72


# Find which was the most popular category each year

## Spark

In [0]:
# create a window function partitioned by post_year and ordered by descending order
window = Window.partitionBy("post_year").orderBy(F.desc("category_count"))

# join df_pin and df_geo and find the most popular category each year
popular_category_each_year = (
    df_pin.join(df_geo, on=df_pin.ind == df_geo.ind, how="inner")
    .withColumn("post_year", F.year(F.col("timestamp")))
    .select(F.col("post_year"), F.col("category"))
    .groupBy("post_year", "category").agg(F.count("*").alias("category_count"))
    .withColumn("rank", F.rank().over(window))
    .filter(F.col("rank") == 1)
    .drop(F.col("rank"))
)

popular_category_each_year.show()

+---------+----------+--------------+
post_year| category|category_count|
+---------+----------+--------------+
 2017|home-decor| 42|
 2018| christmas| 210|
 2019| christmas| 205|
 2020| christmas| 191|
 2021| education| 200|
 2022| christmas| 171|
+---------+----------+--------------+

## SQL

In [0]:
%sql
SELECT post_year, category, category_count
FROM( 
  SELECT YEAR(timestamp) as post_year,
        category,
        COUNT(*) as category_count,
        RANK() OVER (PARTITION BY YEAR(timestamp) ORDER BY COUNT(*) DESC) AS rank
  FROM view_pin
  INNER JOIN view_geo ON view_pin.ind = view_geo.ind
  GROUP BY YEAR(TIMESTAMP), category
) tmp
WHERE rank = 1

post_year,category,category_count
2017,home-decor,42
2018,christmas,210
2019,christmas,205
2020,christmas,191
2021,education,200
2022,christmas,171


## Find the user with most followers in each country

## Spark

In [0]:
# window function partitioned by "country" and ordered by "follower_count" so that it can be ranked later
window = Window.partitionBy("country").orderBy(F.desc("follower_count"))

country_with_the_most_follower_count = (
    df_geo.join(df_pin, on="ind", how="inner")
            .join(df_user, on=df_geo.ind == df_user.ind, how="inner")
            .select("country", "user_name", "poster_name", "follower_count")
            .withColumn("rank", F.row_number().over(window))
            .filter(F.col("rank") == 1)
            .orderBy(F.desc(F.col("follower_count")))
            .limit(1)
            .select("country", "follower_count")
)

country_with_the_most_follower_count.show()

+--------+--------------+
 country|follower_count|
+--------+--------------+
Anguilla| 15000000|
+--------+--------------+

## SQL

In [0]:
%sql
SELECT country, follower_count
FROM (SELECT country, 
      user_name, 
      poster_name, 
      follower_count,
      ROW_NUMBER() OVER (PARTITION BY country ORDER BY follower_count DESC) as rank
FROM view_geo
INNER JOIN view_pin ON view_geo.ind = view_pin.ind
INNER JOIN view_user ON view_geo.ind = view_user.ind) tmp
WHERE rank = 1
ORDER BY follower_count DESC
LIMIT 1

country,follower_count
Anguilla,15000000


# Find the most popular category for different age groups

## Spark

In [0]:
# window function partitioned by 'age_group' and ordered by 'count' to rank the count
window = Window.partitionBy("agE_group").orderBy(F.desc("category_count"))

most_popular_category_by_age_group = (
    df_user.withColumn(
        "age_group",
        F.when((F.col("age") >= 18) & (F.col("age") <= 24), "18-24")
        .when((F.col("age") >= 25) & (F.col("age") <= 35), "25-35")
        .when((F.col("age") >= 36) & (F.col("age") <= 50), "36-50")
        .otherwise("+50")
    )
    .join(df_pin, on="ind", how="inner")
    .groupBy("age_group", "category").agg(F.count("*").alias("category_count"))
    .withColumn("rank", F.row_number().over(window))
    .filter(F.col("rank") == 1)
    .drop("rank")
)

display(most_popular_category_by_age_group)

age_group,category,category_count
+50,vehicles,114
18-24,tattoos,615
25-35,christmas,321
36-50,vehicles,215


## SQL

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW age_group_table AS
SELECT ind, age, 
        CASE
          WHEN age BETWEEN 18 AND 24 THEN "18-24"
          WHEN age BETWEEN 25 AND 35 THEN "25-35"
          WHEN age BETWEEN 36 AND 50 THEN "36-50"
          ELSE "+50"
        END AS age_group
 FROM view_user;

CREATE OR REPLACE TEMPORARY VIEW age_group_count_table AS
SELECT category, 
        age_group, 
        COUNT(*) AS category_count,
        ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY COUNT(*) DESC) as rank
FROM view_pin
INNER JOIN age_group_table ON view_pin.ind = age_group_table.ind
GROUP BY age_group, category;

SELECT age_group, category, category_count
FROM age_group_count_table
WHERE rank = 1;

age_group,category,category_count
+50,vehicles,114
18-24,tattoos,615
25-35,christmas,321
36-50,vehicles,215


# Find the median follower count for different age groups

## Spark

In [0]:
median_follower_count_for_different_age_groups = (
    df_user.withColumn(
        "age_group",
        F.when(
            (F.col("age") >= 18) & (F.col("age") <= 24), "18-24"
        )
        .when((F.col("age") >= 25) & (F.col("age") <= 35), "25-35")
        .when((F.col("age") >= 36) & (F.col("age") <= 50), "36-50")
        .otherwise("+50")
    )
    .join(df_pin, "ind", "inner")
    .select("age_group", "follower_count")
    .dropna("any")
    .withColumn(
        "rank",
        F.row_number().over(Window.partitionBy("age_group").orderBy(F.asc("follower_count")))
    )
    .withColumn(
        "total",
        F.count("age_group").over(Window.partitionBy("age_group"))
    )
    .filter(
        (F.col("rank") == F.col("total")/2) | (F.col("rank") == (F.col("total") +1)/2)
    )
    .select("age_group", "follower_count")
    .withColumnRenamed("follower_count", "median_follower_count")
)

median_follower_count_for_different_age_groups.show()

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 +50| 3000|
 18-24| 130000|
 25-35| 22000|
 36-50| 7000|
+---------+---------------------+

## SQL

In [0]:
%sql
WITH age_grouped AS (
  SELECT 
    CASE 
      WHEN age BETWEEN 18 AND 24 THEN '18-24'
      WHEN age BETWEEN 25 AND 35 THEN '25-35' 
      WHEN age BETWEEN 36 AND 50 THEN '36-50'
      ELSE '+50'
    END AS age_group,
    p.follower_count
  FROM view_user u
  JOIN view_pin p ON u.ind = p.ind
  WHERE p.follower_count IS NOT NULL
),

ranked_data AS (
  SELECT age_group, follower_count,
          row_number() OVER (PARTITION BY age_group ORDER BY follower_count ASC) AS rank,
          COUNT(age_group) OVER (PARTITION BY age_group) AS total
  FROM age_grouped
)

SELECT age_group, follower_count AS median_follower_count
FROM ranked_data
WHERE rank = total/2 OR rank = (total+1)/2

age_group,median_follower_count
+50,3000
18-24,130000
25-35,22000
36-50,7000


# Find how many users have joined each year?

## Spark

In [0]:
display(
    df_user.dropDuplicates(["user_name"])
    .withColumn("post_year", F.year(F.col("date_joined")))
    .select("post_year")
    .groupBy("post_year")
    .agg(F.count("*").alias("number_users_joined"))
)

post_year,number_users_joined
2015,796
2016,2643
2017,1642


## SQL

In [0]:
%sql
WITH distinct_users AS(
  SELECT *
  FROM (
    SELECT *,
          row_number() OVER (PARTITION BY user_name ORDER BY date_joined ASC) AS rank
    FROM view_user
  )
  WHERE rank = 1
),

year_table AS (
  SELECT YEAR(date_joined) as post_year,
          COUNT(*) AS number_users_joined
  FROM distinct_users
  GROUP BY post_year
)

SELECT * FROM year_table

post_year,number_users_joined
2015,826
2016,2650
2017,1605


# Find the median follower count of users based on their joining year

## Spark

In [0]:
median_follower_count_each_year = (
    df_pin.join(df_user, "ind", "inner")
    .withColumn("post_year", F.year(F.col("date_joined")))
    .select("post_year", "follower_count")
    .dropna("any")
    .withColumn(
        "rank",
        F.row_number().over(Window.partitionBy("post_year").orderBy(F.asc("follower_count")))
    )
    .withColumn(
        "total",
        F.count("follower_count").over(Window.partitionBy("post_year"))
    )
    .filter(
        (F.col("rank") == F.col("total")/2) | (F.col("rank") == (F.col("total")+1)/2)
    )
    .select("post_year", "follower_count")
    .withColumnRenamed("follower_count", "median_follower_count")
)

display(median_follower_count_each_year)

post_year,median_follower_count
2015,160000
2016,20000
2017,4000


## SQL

In [0]:
%sql
WITH joined_table AS (
  SELECT YEAR(date_joined) AS post_year,
          follower_count
  FROM view_pin p
  INNER JOIN view_user u
  ON p.ind = u.ind
  WHERE follower_count IS NOT NULL
),

rank_total_table AS (
  SELECT *,
          row_number() OVER (PARTITION BY post_year ORDER BY follower_count ASC) AS rank,
          COUNT(follower_count) OVER (PARTITION BY post_year) AS total
  FROM joined_table 
)

SELECT post_year, follower_count AS median_follower_count
FROM rank_total_table
WHERE rank=total/2 OR rank=(total+1)/2

post_year,median_follower_count
2015,160000
2016,20000
2017,4000


# Find the median follower count of users based on their joining year and age group

## Spark

In [0]:
display(
    df_user.withColumn(
        "age_group",
        F.when(
            (F.col("age") >= 18) & (F.col("age") <= 24), "18-24"
        )
        .when((F.col("age") >= 25) & (F.col("age") <= 35), "25-35")
        .when((F.col("age") >= 36) & (F.col("age") <= 50), "36-50")
        .otherwise("+50")
    )
    .withColumn(
        "post_year",
        F.year(F.col("date_joined"))
    )
    .join(df_pin, "ind", "inner")
    .dropna("any", subset=["follower_count"])
    .select("post_year", "age_group", "follower_count")
    .withColumn(
        "rank",
        F.row_number().over(Window.partitionBy("post_year", "age_group").orderBy("follower_count"))
    )
    .withColumn(
        "total",
        F.count("follower_count").over(Window.partitionBy("post_year", "age_group"))
    )
    .filter(
        (F.col("rank") == F.col("total")/2) | (F.col("rank") == (F.col("total")+1)/2)
    )
    .select("post_year", "age_group", "follower_count")
    .withColumnRenamed("follower_count", "median_follower_count")
)

post_year,age_group,median_follower_count
2015,+50,11000
2015,18-24,375000
2015,25-35,37000
2015,36-50,22000
2016,+50,3000
2016,18-24,46000
2016,25-35,24000
2016,36-50,9000
2017,+50,3000
2017,18-24,7000


## SQL

In [0]:
%sql
WITH age_grouped AS (
  SELECT *,
        CASE 
          WHEN age BETWEEN 18 AND 24 THEN '18-24'
          WHEN age BETWEEN 25 AND 35 THEN '25-35'
          WHEN age BETWEEN 36 AND 50 THEN '36-50'
          ELSE '+50'
        END AS age_group,
        YEAR(date_joined) AS post_year
  FROM view_user u
  INNER JOIN view_pin p
  ON u.ind = p.ind
  WHERE follower_count IS NOT NULL
),

ranked_total_data AS (
  SELECT *,
        row_number() OVER (PARTITION BY post_year, age_group ORDER BY follower_count ASC) AS rank,
        COUNT(follower_count) OVER (PARTITION BY post_year, age_group) AS total
  FROM age_grouped
)

SELECT post_year, age_group, follower_count AS median_follower_count
FROM ranked_total_data
WHERE rank = total/2 OR rank = (total+1)/2

post_year,age_group,median_follower_count
2015,+50,11000
2015,18-24,375000
2015,25-35,37000
2015,36-50,22000
2016,+50,3000
2016,18-24,46000
2016,25-35,24000
2016,36-50,9000
2017,+50,3000
2017,18-24,7000
